In [24]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [2]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [3]:
df = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv').sample(100000)

df = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)

df = df[~df['review_body'].isnull()]

df['review_body'] = df['review_headline'] + '. ' + df['review_body']

df["health_hazard"] = np.nan

df = df.set_index("review_id")

In [4]:
df

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard
review_id,,,,,,,,,,,,
R2R34ZZPB6S62B,B00FQGP20Q,"Barilla Pasta Sauce Variety Pack, 24 Ounce, 4 ...",1,2,9,N,Y,One Star,One Star. to tomato taste,0.000000,0.000000,NaN
R1JUMWWDBPRQ2R,B000E7L2S8,Certified Organic Hard Red Wheat Sprouting See...,4,0,0,N,Y,Good product,Good product. This seeds produced a good crop ...,0.132000,0.567639,NaN
R3LSN6GUHX3U6E,B004M13JUG,"Indian Summer Apple Sauce, Natural, 48-Ounce C...",5,0,0,N,Y,Our favorite brand...,Our favorite brand.... ..No added sugar or jun...,0.256548,0.428869,NaN
R3PIURZUDEESG2,B001BM4JLC,Earth's Best Organic Soy Infant Formula with Iron,5,0,0,N,Y,Five Stars,Five Stars. MY BABY LIKE IT SO MUCH!,0.250000,0.200000,NaN
R17IW8PK20QINJ,B0026WHP80,Oh Boy! Oberto Classics Peppered Thin Style Be...,3,0,0,N,Y,but they are my favorite..,but they are my favorite... I keep getting one...,0.166667,0.333333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
R3NGCJT4YXXZWG,B0018QYMMW,"LesserEvil Classic Kettle Corn, 6.5-Ounce Bags...",4,0,0,N,Y,Pretty good,Pretty good. I waffled between giving this ket...,-0.004514,0.393866,NaN
R3FYOW81RIBZ54,B00H7EOWY2,Kullanard-Durian Wafers Premium Product of Tha...,5,0,0,N,Y,Five Stars,Five Stars. Very good,0.910000,0.780000,NaN
RZTUW2VQ193JM,B000FMTJJQ,"Madhava Organic, Agave Nectar - Light, 11.75-O...",1,1,6,N,N,Can't Open,Can't Open. The package of the product is appe...,0.085417,0.395833,NaN


In [5]:
neg_reviews = df.query("star_rating < 4", engine="python")

In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

#neg_reviews["tokened_review"] = np.nan
all_tokens = []
#neg_reviews.astype({"tokened_review": 'object'}).dtypes
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        token = stemmer.stem(token)
        if token not in stop_words:
            new_text.append(token)
    all_tokens.append(new_text)
neg_reviews["tokened_review"] = all_tokens

<ipython-input-6-eba5c76ea0a6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["tokened_review"] = all_tokens


In [7]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R2R34ZZPB6S62B,B00FQGP20Q,"Barilla Pasta Sauce Variety Pack, 24 Ounce, 4 ...",1,2,9,N,Y,One Star,One Star. to tomato taste,0.000000,0.000000,NaN,"[one, star, tomato, tast]"
R17IW8PK20QINJ,B0026WHP80,Oh Boy! Oberto Classics Peppered Thin Style Be...,3,0,0,N,Y,but they are my favorite..,but they are my favorite... I keep getting one...,0.166667,0.333333,NaN,"[favorit, keep, get, one, packag, bunch, total..."
R2VHUX2QG8CJTT,B00473Q85C,"Regal Picon Cheese Spread, 5-Ounce (Pack of 3)",1,1,4,N,Y,Horrible,"Horrible. This is usually a good product, but ...",0.140000,0.420000,NaN,"[horribl, thi, usual, good, product, wa, sent,..."
RENA8FWZ4D15W,B00DPOEEFU,Golden Eagle Herbal Chew NonTobacco Chews Wint...,1,0,0,N,Y,Buhhhh,Buhhhh. tastes like nasty,-1.000000,1.000000,NaN,"[buhhhh, tast, like, nasti]"
RALRO68ICWNLO,B003EMSGW0,Jelly Belly Gift Box - 10-Flavor - 4.25 oz.,1,3,3,N,Y,A very small box of candy for too much money,A very small box of candy for too much money. ...,0.003125,0.325000,NaN,"[veri, small, box, candi, much, money, tini, l..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R338LOM5OYTV99,B001LM5V7S,Mt. Vikos Glazed & Roasted Fig Spreads - Hazel...,3,0,0,N,Y,Mmm,Mmm. It wasn't my favorite flavor but it did h...,0.525000,0.800000,NaN,"[mmm, wasnt, favorit, flavor, nice, fignutti, ..."
R24IASHZSE543G,B004DDEQAU,Russell Stover Assorted Chocolate Caramels & C...,1,1,1,N,Y,"Old, stale, chocolates missing.","Old, stale, chocolates missing.. Chocolates we...",0.095000,0.351250,NaN,"[old, stale, chocol, miss, chocol, old, tast, ..."
R2SU0F2CT0OL1B,B005VEKDIE,"Assorted Chocolate Candy ""Roshen""",1,3,16,N,N,supported by terrorist,supported by terrorist. Don't ever buy any Ros...,0.104167,0.125000,NaN,"[support, terrorist, dont, ever, buy, ani, ros..."


In [8]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale", "contaminate",'disease', 'disgusting', 'hospital', 'poisoning', 'hazardous', 'ill', 'mold','moldy', 'mildew', 'dangerous' 'rancid', 'smelly', 'stale', 'unhealthy', 'sick','expired','expiration']
for word in hazardous_words:
    neg_reviews.loc[[stemmer.stem(word) in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

C:\Users\MattW\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [9]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

1976

In [10]:
non_hazardous = neg_reviews.query("health_hazard != 1", engine="python").sample(3*num_labeled)
for i in list(non_hazardous.index):
    neg_reviews.loc[i, "health_hazard"] = 0

C:\Users\MattW\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [11]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values

In [12]:
pprint.pprint(reviews)

array([list(['one', 'star', 'tomato', 'tast']),
       list(['favorit', 'keep', 'get', 'one', 'packag', 'bunch', 'total', 'gross', 'moldi', 'contact', 'oberto', 'compani', 'sever', 'time', 'ha', 'respons', 'ive', 'stop', 'buy', 'favorit']),
       list(['horribl', 'thi', 'usual', 'good', 'product', 'wa', 'sent', 'rotten', 'product', 'wa', 'decompos', 'smell', 'went', 'garbag', 'sinc', 'odor', 'wa', 'overwhelm', 'demand', 'refund', 'ripoff']),
       ...,
       list(['mmm', 'wasnt', 'favorit', 'flavor', 'nice', 'fignutti', 'tast', 'thi', 'opinion', 'may', 'love']),
       list(['old', 'stale', 'chocol', 'miss', 'chocol', 'old', 'tast', 'stale', 'start', 'turn', 'white', 'dri', 'mani', 'caramel', 'label', 'bought', 'thi', 'specif', 'raspberri', 'caramel', 'entir', '3', 'box', 'none', 'interior', 'label', 'say', '6', 'buy']),
       list(['support', 'terrorist', 'dont', 'ever', 'buy', 'ani', 'roshen', 'product', 'owner', 'thi', 'compani', 'ukranian', 'terrorist', 'time', 'ukranian', 'pre

In [13]:
for i in range(len(reviews)):
    reviews[i] = ' '.join(reviews[i])

In [14]:
reviews

array(['one star tomato tast',
       'favorit keep get one packag bunch total gross moldi contact oberto compani sever time ha respons ive stop buy favorit',
       'horribl thi usual good product wa sent rotten product wa decompos smell went garbag sinc odor wa overwhelm demand refund ripoff',
       ...,
       'mmm wasnt favorit flavor nice fignutti tast thi opinion may love',
       'old stale chocol miss chocol old tast stale start turn white dri mani caramel label bought thi specif raspberri caramel entir 3 box none interior label say 6 buy',
       'support terrorist dont ever buy ani roshen product owner thi compani ukranian terrorist time ukranian presid petro poroshenko awar'],
      dtype=object)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix = TfidfVectorizer(max_features=200)
X = matrix.fit_transform(reviews).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
from sklearn.naive_bayes import GaussianNB
classifier_g = GaussianNB()

from sklearn.linear_model import LogisticRegression

classifier_l = LogisticRegression()

classifier_g.fit(X_train, y_train)
classifier_l.fit(X_train, y_train)

# Predict Class
y_pred_g = classifier_g.predict(X_test)
y_pred_l = classifier_l.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy_g = accuracy_score(y_test, y_pred_g)
accuracy_l = accuracy_score(y_test, y_pred_l)
print(accuracy_g)

0.9387651821862348


In [18]:
trial = neg_reviews.sample(100)

In [19]:
matrix.get_feature_names()

['actual',
 'ad',
 'almost',
 'also',
 'amazon',
 'ani',
 'anoth',
 'arriv',
 'away',
 'back',
 'bad',
 'bag',
 'bar',
 'bean',
 'becaus',
 'befor',
 'best',
 'better',
 'big',
 'bit',
 'bitter',
 'bottl',
 'bought',
 'box',
 'brand',
 'broken',
 'butter',
 'buy',
 'came',
 'candi',
 'cant',
 'chang',
 'chip',
 'chocol',
 'coffe',
 'come',
 'compani',
 'contain',
 'cook',
 'cooki',
 'could',
 'cup',
 'date',
 'day',
 'didnt',
 'differ',
 'disappoint',
 'disgust',
 'doe',
 'doesnt',
 'dont',
 'dri',
 'drink',
 'eat',
 'enjoy',
 'enough',
 'even',
 'ever',
 'expect',
 'expens',
 'expir',
 'feel',
 'find',
 'fine',
 'first',
 'flavor',
 'food',
 'found',
 'free',
 'fresh',
 'get',
 'gift',
 'give',
 'go',
 'good',
 'got',
 'great',
 'green',
 'ha',
 'half',
 'hard',
 'high',
 'hope',
 'horribl',
 'hot',
 'howev',
 'ill',
 'im',
 'ingredi',
 'instead',
 'item',
 'ive',
 'kind',
 'know',
 'label',
 'last',
 'less',
 'like',
 'list',
 'littl',
 'local',
 'look',
 'lot',
 'love',
 'made',
 'm

In [20]:
trial = neg_reviews
trial_x = trial.loc[:,"tokened_review"].values
for i in range(len(trial_x)):
    trial_x[i] = ' '.join(trial_x[i])
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier_l.predict(trial_X)
trial_y

array([0., 0., 0., ..., 0., 0., 0.])

In [21]:
neg_reviews["health_hazard"] = trial_y

<ipython-input-21-56d181f868ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["health_hazard"] = trial_y


In [22]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R2R34ZZPB6S62B,B00FQGP20Q,"Barilla Pasta Sauce Variety Pack, 24 Ounce, 4 ...",1,2,9,N,Y,One Star,One Star. to tomato taste,0.000000,0.000000,0.0,one star tomato tast
R17IW8PK20QINJ,B0026WHP80,Oh Boy! Oberto Classics Peppered Thin Style Be...,3,0,0,N,Y,but they are my favorite..,but they are my favorite... I keep getting one...,0.166667,0.333333,0.0,favorit keep get one packag bunch total gross ...
R2VHUX2QG8CJTT,B00473Q85C,"Regal Picon Cheese Spread, 5-Ounce (Pack of 3)",1,1,4,N,Y,Horrible,"Horrible. This is usually a good product, but ...",0.140000,0.420000,0.0,horribl thi usual good product wa sent rotten ...
RENA8FWZ4D15W,B00DPOEEFU,Golden Eagle Herbal Chew NonTobacco Chews Wint...,1,0,0,N,Y,Buhhhh,Buhhhh. tastes like nasty,-1.000000,1.000000,0.0,buhhhh tast like nasti
RALRO68ICWNLO,B003EMSGW0,Jelly Belly Gift Box - 10-Flavor - 4.25 oz.,1,3,3,N,Y,A very small box of candy for too much money,A very small box of candy for too much money. ...,0.003125,0.325000,0.0,veri small box candi much money tini littl pac...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R338LOM5OYTV99,B001LM5V7S,Mt. Vikos Glazed & Roasted Fig Spreads - Hazel...,3,0,0,N,Y,Mmm,Mmm. It wasn't my favorite flavor but it did h...,0.525000,0.800000,0.0,mmm wasnt favorit flavor nice fignutti tast th...
R24IASHZSE543G,B004DDEQAU,Russell Stover Assorted Chocolate Caramels & C...,1,1,1,N,Y,"Old, stale, chocolates missing.","Old, stale, chocolates missing.. Chocolates we...",0.095000,0.351250,1.0,old stale chocol miss chocol old tast stale st...
R2SU0F2CT0OL1B,B005VEKDIE,"Assorted Chocolate Candy ""Roshen""",1,3,16,N,N,supported by terrorist,supported by terrorist. Don't ever buy any Ros...,0.104167,0.125000,0.0,support terrorist dont ever buy ani roshen pro...


In [23]:
dangerous = neg_reviews.query("health_hazard == 1", engine="python")
dangerous

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R2NZM3E2KU1LO2,B004JRSAJS,"Miracle Noodle Shirataki Rice, 8-Ounce (Pack o...",1,0,1,N,N,Confused,Confused. I was really excited to try the mira...,-0.187500,0.531250,1.0,confus wa realli excit tri miracl product wa 5...
R3Q9FFD0I1PR83,B00XLM01W6,Reppert's Candies Coconut Strips Coconut Slice...,1,1,1,N,Y,NOT FOR THE HEAT OF SUMMER!,NOT FOR THE HEAT OF SUMMER!. This is a reputab...,-0.042857,0.331429,1.0,heat summer thi reput candi manufactur howev o...
RNFDREO6G6M6D,B0060KOF7I,Crazy Cups Roast Sampler,2,0,0,N,Y,MOSTLY DECAF,MOSTLY DECAF. 1. It took 10 days fromt he time...,-0.100000,0.193590,1.0,mostli decaf 1 took 10 day fromt time order wa...
R2QURMMFEW3J0J,B001FXZ5L0,Linden's Butter Crunch Cookies 3 Cookies Per P...,2,0,0,N,Y,Mixed feelings for this company,Mixed feelings for this company. Most of my co...,0.159762,0.343095,1.0,mix feel thi compani cooki veri soft stale thi...
RK1VZP20FMUQQ,B001YJBLMM,"Crunchmaster Multi-Grain Crackers, Gluten Free...",2,0,0,N,Y,"I eat these crackers all the time, and the ...","I eat these crackers all the time, and the ......",-0.288194,0.486111,1.0,eat cracker time eat cracker time one came ama...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R2PFNEN9SCWT39,B003VMY3GQ,"Bull's Eye Original Barbecue Sauce, 18 Ounce B...",1,2,4,N,N,Just bought Bull's Eye Original Barbecue Sauce,Just bought Bull's Eye Original Barbecue Sauce...,-0.035220,0.463157,1.0,bought bull eye origin barbecu sauc wow alway ...
REKUX7B27WA40,B00113HA3A,Good Earth Organic Tea,1,0,0,N,N,Tastes horrible,Tastes horrible. I bought this tea to serve at...,0.050000,0.075000,1.0,tast horribl bought thi tea serv gather wa emb...
R11EYZ3V27FFRS,B0025UCHT0,"Barilla Plus Pasta, 14.5 Ounce",1,0,0,N,N,Dont know whats wrong with this stuff,Dont know whats wrong with this stuff. I ate t...,0.055000,0.373333,1.0,dont know wrong thi stuff ate thi pasta day ag...


In [25]:
def plot_box(y1,y2,y3,y4,y5, xtitle, ytitle):


    fig = go.Figure()
    
    fig.add_trace(go.Box(y = y1,name='1 Star',boxpoints=False))
    fig.add_trace(go.Box(y = y2,name='2 Star',boxpoints=False))
    fig.add_trace(go.Box(y = y3,name='3 Star',boxpoints=False))
    fig.add_trace(go.Box(y = y4,name='4 Star',boxpoints=False))
    fig.add_trace(go.Box(y = y5,name='5 Star',boxpoints=False))
    
    fig.update_layout(
        xaxis_title=xtitle,
        yaxis_title=ytitle
        )
    
    return fig

In [25]:
#LDA
data_filtered = dangerous['review_body'].values
pprint.pprint(data_filtered)

array(["Not as good as their other products. I've tried the Peach green tea by Steaz and thought this would taste similar, but I think something is lose in translation as far as how this product is zero calories. Not as sweet or full-flavored. Love the brand, just not this drink.",
       'Not very tasty. Was hoping, given the reviews, for something a little tastier.  Very fruit & seed heavy and tastes of it.  Imagine dried cranberries and sunflower seeds.  My sister has nut allergies, so I was looking for something she would like, but I think I am going to have to keep looking',
       "Didn't order decaf. I received 6 decaf K Cups. I don't drink decaf so it was a waste to a degree. However the rest if the coffee was great. If I had know decaf was in the package I would have purchased something else.",
       ...,
       "I love these bars, but they taste stale from this supplier. First of all, I'd like to begin by saying that I love anything that has chocolate and raspberry flavoring

In [47]:
polarity_hazard_rev=dangerous['star_rating'].values
plot_box(y1 = polarity_hazard_rev,
        y2 = [],
        y3 = [],
        y4 = [],
        y5 = [],
        xtitle='',
        ytitle='Review Length')


In [26]:
def sent_to_words(sentences):
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = sentence.translate(translator)
        yield(nltk.word_tokenize(str(sentence)))

data_words = list(sent_to_words(data_filtered))
clean_data = []
for i in range(len(data_words)):
    clean_review = []
    for j in range(len(data_words[i])):
        if data_words[i][j] not in stop_words:
            clean_review.append(data_words[i][j])
    clean_data.append(clean_review)

In [27]:
# Create Dictionary
id2word = corpora.Dictionary(clean_data)
# Create Corpus
texts = clean_data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
pprint.pprint(corpus[:5])

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1)],
 [(14, 2),
  (19, 1),
  (23, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1)],
 [(2, 1),
  (14, 1),
  (23, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 4),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1)],
 [(14, 1),
  (20, 1),
  (23, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 2),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1)],
 [(2, 2),
  (5, 3),
  (17, 1),
  (30, 1),
  (34, 1),
  (35, 1),
  (52, 1

In [28]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       minimum_probability=0.1,
                                       workers=3,
                                       passes=2)
# Print the Keyword in the 10 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"like" + 0.009*"coffee" + 0.008*"taste" + 0.007*"product" + '
  '0.006*"something" + 0.006*"one" + 0.006*"would" + 0.006*"good" + '
  '0.006*"different" + 0.005*"flavor"'),
 (1,
  '0.010*"product" + 0.007*"coffee" + 0.006*"something" + 0.006*"tea" + '
  '0.006*"flavor" + 0.005*"taste" + 0.005*"good" + 0.005*"one" + 0.005*"ill" + '
  '0.005*"get"'),
 (2,
  '0.009*"like" + 0.008*"taste" + 0.007*"product" + 0.007*"good" + 0.007*"one" '
  '+ 0.006*"different" + 0.006*"flavor" + 0.006*"something" + 0.005*"even" + '
  '0.004*"great"'),
 (3,
  '0.013*"like" + 0.010*"something" + 0.010*"taste" + 0.008*"good" + '
  '0.007*"one" + 0.007*"would" + 0.006*"different" + 0.006*"dont" + '
  '0.006*"coffee" + 0.006*"ill"'),
 (4,
  '0.014*"like" + 0.010*"taste" + 0.008*"something" + 0.008*"different" + '
  '0.007*"would" + 0.006*"tea" + 0.005*"one" + 0.005*"product" + 0.005*"good" '
  '+ 0.005*"sugar"')]


In [29]:
#from itertools import chain
#lda_corpus = lda_model[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
#scores = list(chain(*[[score for topic_id,score in topic] for topic in [doc for doc in lda_corpus]]))
#threshold = sum(scores)/len(scores)


In [30]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models
data_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(data_vis)